# 🌍🌾 FOOD PRICE SHOCK & HUNGER RISK ANALYTICS  
### *An End-to-End Real-World Data Engineering, Analytics & Decision Intelligence Project*

---

##  PROBLEM STATEMENT
Sudden food price inflation caused by climate shocks, geopolitical conflicts, fuel costs, and supply chain disruptions pushes vulnerable populations into hunger and food insecurity. Despite the availability of large-scale international data, governments and humanitarian agencies lack an integrated analytics system to identify early warning signals of hunger risk. This results in delayed interventions and increased human suffering.

---

##  AIM
To design and implement a data-driven analytics framework that identifies food price shocks and evaluates their impact on hunger risk across countries using real-world, messy datasets.

---

##  OBJECTIVES
- To ingest and clean large-scale, unstructured real-world food price data  
- To perform advanced data cleaning and feature engineering using Python  
- To conduct SQL-based analytical queries within a cloud environment  
- To quantify food price volatility and hunger risk using engineered indices  
- To generate analytics-ready datasets for dashboarding and decision support  

---

##  DATA DESCRIPTION
This project uses a real-world, uncleaned dataset simulating FAO and World Bank food price records combined with hunger indicators across multiple countries and years. The data contains missing values, inconsistent text formats, outliers, and time-series noise — reflecting true industry data conditions.

---

##  TOOLS & TECHNOLOGIES
- Google Colab (Cloud-based analytics environment)  
- Python (Pandas, NumPy) for data cleaning and feature engineering  
- SQLite for structured SQL analytics within Colab  
- CSV / Excel outputs for dashboard integration  
- Power BI / Tableau for analytical dashboards  
- AI Application layer for insight explanation and decision support  

---

##  METHODOLOGY
1. Raw dataset ingestion and schema inspection  
2. Comprehensive data cleaning and normalization  
3. Handling missing values using statistical imputation  
4. Outlier detection and removal using IQR method  
5. Feature engineering (Price Shock Index, Hunger Risk Level)  
6. SQL-based aggregation and analytical querying  
7. Export of clean, analytics-ready datasets  

---

## KEY ANALYTICS PERFORMED
- Country-wise average food prices  
- Commodity-wise inflation trends  
- Detection of sudden food price shocks  
- Identification of high hunger-risk countries  
- Temporal analysis of price vs hunger indicators  

---

##  EXPECTED OUTCOMES
- Identification of countries vulnerable to food price shocks  
- Early warning signals for rising hunger risk  
- Clean datasets suitable for BI dashboards and AI applications  
- Policy-ready insights for governments and NGOs  

---

##  SIGNIFICANCE OF THE PROJECT
This project goes beyond traditional business analytics by addressing a global humanitarian challenge. It demonstrates real-world data engineering skills, advanced analytics capability, and the ability to translate raw data into actionable insights. The framework can be extended into predictive systems for proactive food security planning.

---

##  FUTURE ENHANCEMENTS
- Integration of climate and conflict datasets  
- Predictive modeling for hunger risk forecasting  
- Real-time data pipelines  
- Deployment as a full-stack AI-powered decision support system  

---

###  *This project reflects real industry practices in data analytics, policy intelligence, and ethical data science.*


In [4]:
from google.colab import files

import pandas as pd

import numpy as np



# Upload file from computer
uploaded = files.upload()

file_name = list(uploaded.keys())[0]

df = pd.read_csv(file_name)



# TEXT STANDARDIZATION
df['Country'] = df['Country'].astype(str).str.strip().str.title()

df['Commodity'] = df['Commodity'].replace(" ", np.nan)

df['Commodity'] = df['Commodity'].fillna("Unknown")

df['Market_Type'] = df['Market_Type'].fillna("Unknown")



# DATE CLEANING
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df = df.dropna(subset=['Date'])



# NUMERIC CONVERSION
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

df['Inflation_Rate'] = pd.to_numeric(df['Inflation_Rate'], errors='coerce')

df['Hunger_Index'] = pd.to_numeric(df['Hunger_Index'], errors='coerce')



# NULL VALUE IMPUTATION
df['Price'] = (
    df.groupby(['Country', 'Commodity'])['Price']
      .transform(lambda x: x.fillna(x.median()))
)

df['Inflation_Rate'] = (
    df.groupby('Country')['Inflation_Rate']
      .transform(lambda x: x.fillna(x.mean()))
)

df['Hunger_Index'] = (
    df.groupby('Country')['Hunger_Index']
      .transform(lambda x: x.fillna(x.mean()))
)



# OUTLIER REMOVAL USING IQR
Q1 = df['Price'].quantile(0.25)

Q3 = df['Price'].quantile(0.75)

IQR = Q3 - Q1

df = df[
    (df['Price'] >= Q1 - 1.5 * IQR) &
    (df['Price'] <= Q3 + 1.5 * IQR)
]



# FEATURE ENGINEERING
df['Year'] = df['Date'].dt.year

df['Month'] = df['Date'].dt.month

df['Price_Shock_Index'] = (
    df.groupby(['Country', 'Commodity'])['Price']
      .pct_change() * 100
)



def hunger_risk(value):
    if value < 15:
        return "Low"
    elif value < 30:
        return "Medium"
    else:
        return "High"



df['Hunger_Risk_Level'] = df['Hunger_Index'].apply(hunger_risk)



print("DATA CLEANING COMPLETED")

print("FINAL SHAPE:", df.shape)

df.head()


Saving global_food_price_hunger_raw_messy.csv to global_food_price_hunger_raw_messy (1).csv
DATA CLEANING COMPLETED
FINAL SHAPE: (11870, 12)


,Country,Market_Type,Commodity,Price,Currency,Date,Inflation_Rate,Hunger_Index,Year,Month,Price_Shock_Index,Hunger_Risk_Level
0,Sudan,Urban,Wheat,27.08,NGN,2020-05-22,6.02,31.22,2020,5,NaN,High
1,Ethiopia,Unknown,Wheat,116.39,KES,2015-09-23,6.02,31.22,2015,9,NaN,High
2,Nepal,Urban,Sugar,39.38,INR,2021-02-15,6.02,31.22,2021,2,NaN,High
3,Pakistan,Wholesale,Rice,97.65,SDG,2023-11-29,6.02,31.22,2023,11,NaN,High
4,Sudan,Rural,Maize,112.70,ETB,2021-08-01,6.02,31.22,2021,8,NaN,High


In [5]:
# CELL 2: COMPLETE SQL ANALYSIS

import sqlite3



conn = sqlite3.connect("food_hunger_analysis.db")



df.to_sql("food_data", conn, if_exists="replace", index=False)



# COUNTRY-WISE AVERAGE FOOD PRICE

query_1 = """

SELECT Country,

       ROUND(AVG(Price), 2) AS Avg_Food_Price

FROM food_data

GROUP BY Country

ORDER BY Avg_Food_Price DESC

"""



avg_price_country = pd.read_sql(query_1, conn)



# COMMODITY-WISE YEARLY PRICE TREND

query_2 = """

SELECT Commodity, Year,

       ROUND(AVG(Price), 2) AS Avg_Price

FROM food_data

GROUP BY Commodity, Year

ORDER BY Commodity, Year

"""



commodity_trend = pd.read_sql(query_2, conn)



# HIGH HUNGER RISK COUNTRIES

query_3 = """

SELECT Country,

       ROUND(AVG(Hunger_Index), 2) AS Hunger_Score

FROM food_data

GROUP BY Country

HAVING Hunger_Score > 30

ORDER BY Hunger_Score DESC

"""



high_hunger_countries = pd.read_sql(query_3, conn)



# PRICE SHOCK SEVERITY ANALYSIS

query_4 = """

SELECT Country, Commodity,

       ROUND(AVG(Price_Shock_Index), 2) AS Shock_Level

FROM food_data

GROUP BY Country, Commodity

ORDER BY Shock_Level DESC

"""



price_shock_analysis = pd.read_sql(query_4, conn)



# YEAR-WISE PRICE VS HUNGER RELATIONSHIP

query_5 = """

SELECT Year,

       ROUND(AVG(Price), 2) AS Avg_Price,

       ROUND(AVG(Hunger_Index), 2) AS Avg_Hunger

FROM food_data

GROUP BY Year

ORDER BY Year

"""



price_hunger_trend = pd.read_sql(query_5, conn)



print("SQL ANALYSIS COMPLETED")



avg_price_country.head()


SQL ANALYSIS COMPLETED


,Country,Avg_Food_Price
0,Kenya,101.61
1,India,101.48
2,Nigeria,101.28
3,Yemen,101.09
4,Nepal,100.79


In [6]:
# CELL 3: DOWNLOAD CLEAN DATASET

from google.colab import files



df.to_csv("final_clean_food_hunger_analytics.csv", index=False)

df.to_excel("final_clean_food_hunger_analytics.xlsx", index=False)



files.download("final_clean_food_hunger_analytics.csv")

files.download("final_clean_food_hunger_analytics.xlsx")



print("FILES DOWNLOADED SUCCESSFULLY")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FILES DOWNLOADED SUCCESSFULLY
